In [1]:
#import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")

In [2]:
using JuMP, GLPK
using DataFrames
using CSV

In [3]:
#load cost data
con_generation = CSV.File("../data/conventional_generators.csv") |> DataFrame

#load wind_technicaldata
wind_generation = CSV.File("../data/wind_farms.csv") |> DataFrame

# load wind profile
wind_profile = CSV.File("../data/wind_powerprofile_200.csv") |> DataFrame

# batteries
battery = CSV.File("../data/Battery.csv") |> DataFrame

# transmission_lines
transmission_lines = CSV.File("../data/transmission_lines.csv") |> DataFrame


Row,Transmission lines: From node,To node,Susceptance [per-unit],Capacity [MW]
,Int64,Int64,Float64,Int64
1,1,2,0.0146,175
2,1,3,0.2253,175
3,1,5,0.0907,350
4,2,4,0.1356,175
5,2,6,0.205,175
6,3,9,0.1271,175
7,3,24,0.084,400
8,4,9,0.111,175
9,5,10,0.094,350


In [4]:
#demand bids
name = "demand_hour_0.csv"
demand_bids = CSV.File("../data/demand_bids_hour/" * name) |> DataFrame
 
# number of convential generators
G = size(con_generation, 1)

# number of demand
D = size(demand_bids, 1)

# number of wind generators
W = size(wind_generation, 1)

# number of batteries
B = size(battery, 1)

# number of transmission lines
L = size(transmission_lines, 1)

# number of nodes
N = 24


24

In [5]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[])

# For x variables, manually add each column. This is a one-time setup.
for i in 1:G
    result_df[!, Symbol("x_con$i")] = Float64[]
end

# For w variables, manually add each column. This is a one-time setup.
for i in 1:W
    result_df[!, Symbol("x_wind$i")] = Float64[]
end

# For y variables, manually add each column. This is a one-time setup.
for i in 1:D
    result_df[!, Symbol("y$i")] = Float64[]
end

# For b variables, manually add each column. This is a one-time setup.
for i in 1:B
    result_df[!, Symbol("x_bat_charg$i")] = Float64[]
    result_df[!, Symbol("x_bat_discharg$i")] = Float64[]
end

# For l variables, manually add each column. This is a one-time setup.
for i in 1:N
    result_df[!, Symbol("x_angle$i")] = Float64[]
end



equilibrium_df = DataFrame(hour = Int[])

# for each node a market price column
for i in 1:N
    equilibrium_df[!, Symbol("market_price_node$i")] = Float64[]
end



In [6]:
println((result_df))

0×68 DataFrame
 Row │ hour   x_con1   x_con2   x_con3   x_con4   x_con5   x_con6   x_con7   x_con8   x_con9   x_con10  x_con11  x_con12  x_wind1  x_wind2  x_wind3  x_wind4  y1       y2       y3       y4       y5       y6       y7       y8       y9       y10      y11      y12      y13      y14      y15      y16      y17      x_bat_charg1  x_bat_discharg1  x_bat_charg2  x_bat_discharg2  x_bat_charg3  x_bat_discharg3  x_bat_charg4  x_bat_discharg4  x_bat_charg5  x_bat_discharg5  x_angle1  x_angle2  x_angle3  x_angle4  x_angle5  x_angle6  x_angle7  x_angle8  x_angle9  x_angle10  x_angle11  x_angle12  x_angle13  x_angle14  x_angle15  x_angle16  x_angle17  x_angle18  x_angle19  x_angle20  x_angle21  x_angle22  x_angle23  x_angle24 
     │ Int64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64

In [37]:
# Load demand bids data
demand_bids_all = [CSV.File("../data/demand_bids_hour/demand_hour_$(i-1).csv") |> DataFrame for i in 1:24]

# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for every generator and hour
@variable(model, x_con[1:G, 1:24] >= 0)  # Power output variable for conventional generators
@variable(model, x_wind[1:W, 1:24] >= 0)  # Power output variable for wind generators
@variable(model, y[1:D, 1:24] >= 0)  # Demand variable
@variable(model, bat_char[1:B, 1:24])  # Battery variable when it is charging
@variable(model, bat_disch[1:B, 1:24])  # Battery variable when it is discharging
@variable(model, angle[1:N, 1:24])  # Voltage angles variable

# Add initial values for the battery
init_bat = []
for b in 1:B
    append!(init_bat, 0)
end

# Define the charging and discharging efficiencies of the battery
eff_char = 0.9
eff_disch= 0.9

# Add constraints for each plant
for g in 1:G
    for hour in 1:24
        @constraint(model, x_con[g, hour] <= con_generation[g, 6])  # Set the upper bound
        @constraint(model, x_con[g, hour] >= 0)  # Set the lower bound
    end
end

# Add constraints for each wind farm
for w in 1:W
    for hour in 1:24
        @constraint(model, x_wind[w, hour] <= wind_profile[hour, w+1])  # Set the upper bound
        @constraint(model, x_wind[w, hour] >= 0)  # Set the lower bound
    end
end

# Add constraints for each demand bid
for d in 1:D
    for hour in 1:24
        @constraint(model, y[d, hour] <= demand_bids_all[hour][d, 2])  # Set the upper bound
        @constraint(model, y[d, hour] >= 0)  # Set the lower bound
    end
end

#add constraints for initial con_generation
for g in 1:G
    @constraint(model, x_con[g, 1] <= con_generation[g, 11] + con_generation[g, 9])  # Set the upper bound
    @constraint(model, x_con[g, 1] >= con_generation[g, 11] - con_generation[g, 10])  # Set the lower bound
end

# Add temporary for each generator
for g in 1:G
    for hour in 2:24
        @constraint(model, x_con[g, hour] <= x_con[g, hour - 1] + con_generation[g, 7])  # Set the upper bound
        @constraint(model, x_con[g, hour] >= x_con[g, hour - 1] - con_generation[g, 8])  # Set the lower bound
    end
end

# add Power constraints for battery charging
for b in 1:B
    for hour in 1:24
        @constraint(model, bat_char[b, hour] <= battery[b, 5])  # Set the upper bound
        @constraint(model, bat_char[b, hour] >= 0)  # Set the lower bound 
    end
end

# add Power constraints for battery discharging, with positive values
for b in 1:B
    for hour in 1:24
        @constraint(model, bat_disch[b, hour] <= battery[b, 5])  # Set the upper bound
        @constraint(model, bat_disch[b, hour] >= 0)  # Set the lower bound
    end
end

# add Capacity constraints for battery charging and discharging
# sum all previous battery charging and discharging, they need to be below the battery capacity
for b in 1:B
    for hour in 1:24
        @constraint(model, sum(init_bat[b] + bat_char[b, h]*eff_char - bat_disch[b, h]/eff_disch for h in 1:hour) <= battery[b, 4])  # Set the upper bound     ##Add initial value + for 1 to 24 instead of 2 to 24
        @constraint(model, sum(init_bat[b] + bat_char[b, h]*eff_char - bat_disch[b, h]/eff_disch for h in 1:hour) >= 0)  # Set the lower bound
    end
end

# add an angle reference to 0 for every hour
for hour in 1:24
    @constraint(model, angle[1, hour] == 0)
end

# add constraints for transmission lines assuming a susceptance of 500
for line in 1:L
    for hour in 1:24
        @constraint(model, 450*(angle[transmission_lines[line, 1],hour]-angle[transmission_lines[line, 2],hour]) <= transmission_lines[line, 4])  # Set the upper bound  #half the cap to set the different prices per zone
        @constraint(model, 450*(angle[transmission_lines[line, 1],hour]-angle[transmission_lines[line, 2],hour]) >= -1 * transmission_lines[line, 4])  # Set the lower bound
    end
end

balance = Vector{Any}(undef, N*24)

# add balance constraint for each node and each demand_hour_
for hour in 1:24
    for node in 1:N
        balance[(hour-1)*24+node] = @constraint(model, sum(x_con[g, hour] for g in 1:G if con_generation[g, 2] == node) +
                            sum(x_wind[w, hour] for w in 1:W if wind_generation[w, 2] == node)  +
                            sum(500*(angle[transmission_lines[line, 1],hour]-angle[transmission_lines[line, 2],hour]) for line in 1:L if transmission_lines[line, 2] == node) -
                            sum(bat_char[b, hour] for b in 1:B if battery[b, 2] == node) +
                            sum(bat_disch[b, hour] for b in 1:B if battery[b, 2] == node) 
                            ==
                            sum(y[d, hour] for d in 1:D if demand_bids_all[hour][d, 1] == node) +
                            sum(500*(angle[transmission_lines[line, 1],hour]-angle[transmission_lines[line, 2],hour]) for line in 1:L if transmission_lines[line, 1] == node))
    end
end

# Define the objective function
@objective(model, Max, sum(demand_bids_all[hour][d, 3] * y[d, hour] for d in 1:D, hour in 1:24) - sum(con_generation[g, 3] * x_con[g, hour] for g in 1:G, hour in 1:24) - sum(0.0001 * bat_char[b, hour] for hour in 1:24, b in 1:B) - sum(0.0001 * bat_disch[b, hour] for hour in 1:24, b in 1:B) ) # Add the penalty of battery charging and discharging



# Solve the model
optimize!(model)


# Check the status of the solution
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")

    # RETURN OBJECTIVE value
    println("Objective value: ", objective_value(model))


else
    println("No optimal solution found")
end


Optimal solution found
Objective value: 643753.5470959204


In [28]:
value(angle[2,1])

-0.13255576374749095

In [29]:
# print decision variables
empty!(result_df)
for hour in 1:24
    resultvector = zeros(1+G+W+D+B+B+N)
    resultvector[1] = hour
    for g in 1:G
        resultvector[1+g] = value(x_con[g, hour])
    end
    for w in 1:W
        resultvector[1+G+w] = value(x_wind[w, hour])
    end
    for d in 1:D
        resultvector[1+G+W+d] = value(y[d, hour])
    end
    for b in 1:B
        resultvector[1+G+W+D+2*b-1] = value(bat_char[b, hour])
        resultvector[1+G+W+D+2*b] = value(bat_disch[b, hour])
    end
    for node in 1:N
        resultvector[1+G+W+D+B+B+node] = value(angle[node,hour])
    end
    push!(result_df, resultvector)
end

In [30]:
1+G+W+D+B+B

44

In [31]:
#save the duals of the balance constraints to the equilibrium_df
empty!(equilibrium_df)
for hour in 1:24
    hour_results = []

    for node in 1:N
        push!(hour_results, dual(balance[(hour-1)*24+node]))
    end
    push!(equilibrium_df, [hour; hour_results...])
end




#save results to dataframe
CSV.write("results/optimization_results_marketprices_nodal_angles.csv", equilibrium_df)



"results/optimization_results_marketprices_nodal_angles.csv"

In [32]:
equilibrium_df

Row,hour,market_price_node1,market_price_node2,market_price_node3,market_price_node4,market_price_node5,market_price_node6,market_price_node7,market_price_node8,market_price_node9,market_price_node10,market_price_node11,market_price_node12,market_price_node13,market_price_node14,market_price_node15,market_price_node16,market_price_node17,market_price_node18,market_price_node19,market_price_node20,market_price_node21,market_price_node22,market_price_node23,market_price_node24
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0
2,2,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52
3,3,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52
4,4,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52
5,5,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52,10.52
6,6,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
7,7,14.04,14.04,14.04,14.04,14.04,14.04,14.04,14.04,14.04,14.04,14.04,14.04,14.04,14.04,14.04,14.04,14.04,14.04,14.04,14.04,14.04,14.04,14.04,14.04
8,8,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0
9,9,15.6279,15.6279,15.6279,15.6279,15.6279,15.6279,15.6279,15.6279,15.6279,15.6279,15.6279,15.6279,15.6279,15.6279,15.6279,15.6279,15.6279,15.6279,15.6279,15.6279,15.6279,15.6279,15.6279,15.6279


In [33]:
result_df

Row,hour,x_con1,x_con2,x_con3,x_con4,x_con5,x_con6,x_con7,x_con8,x_con9,x_con10,x_con11,x_con12,x_wind1,x_wind2,x_wind3,x_wind4,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,x_bat_charg1,x_bat_discharg1,x_bat_charg2,x_bat_discharg2,x_bat_charg3,x_bat_discharg3,x_bat_charg4,x_bat_discharg4,x_bat_charg5,x_bat_discharg5,x_angle1,x_angle2,x_angle3,x_angle4,x_angle5,x_angle6,x_angle7,x_angle8,x_angle9,x_angle10,x_angle11,x_angle12,x_angle13,x_angle14,x_angle15,x_angle16,x_angle17,x_angle18,x_angle19,x_angle20,x_angle21,x_angle22,x_angle23,x_angle24
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,0.0,108.5,108.5,280.0,280.0,210.0,217.0,220.232,76.8921,101.54,92.8003,95.3709,67.4817,60.3784,165.153,120.757,197.118,62.1542,207.773,113.653,79.9126,111.878,46.1717,44.3959,85.2401,78.1367,106.55,108.326,120.757,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.132556,0.223728,-0.0970396,0.0437912,-0.179871,-0.338903,-0.18263,0.03082,-0.0267059,0.197113,0.22958,0.252648,0.531691,1.19694,1.10778,1.66998,1.88219,0.754228,0.62798,1.94994,2.01996,0.661557,0.710335
2,2,0.0,0.0,0.0,0.0,0.0,108.5,77.4586,280.0,280.0,210.0,210.027,172.232,66.8277,90.9989,109.167,107.937,63.453,56.7737,155.293,113.547,185.349,58.4435,195.368,106.868,75.1417,105.198,43.4152,41.7454,80.1511,73.4719,100.189,101.859,113.547,15.0,0.0,10.0,0.0,10.0,0.0,3.33333,0.0,5.0,0.0,0.0,-0.129089,0.225939,-0.0948247,0.0300553,-0.178894,-0.326668,-0.179724,0.0262696,-0.0383965,0.187764,0.195911,0.222215,0.540967,1.23064,1.12127,1.70431,1.93815,0.752777,0.598025,2.00273,2.06352,0.593556,0.728289
3,3,0.0,0.0,0.0,0.0,0.0,72.5,72.5,280.0,280.0,210.0,138.027,124.232,78.422,116.959,142.88,134.78,60.4314,54.0702,147.898,108.14,176.523,55.6605,186.065,101.779,71.5635,100.189,41.3478,39.7575,76.3344,69.9732,95.418,97.0083,108.14,15.0,0.0,10.0,0.0,10.0,0.0,20.0,0.0,5.0,0.0,0.0,-0.140053,0.218234,-0.116138,0.0426816,-0.195881,-0.344621,-0.204675,-0.00952787,-0.0690398,0.140067,0.108889,0.121821,0.517016,1.1973,1.11025,1.71305,1.96142,0.694184,0.481679,2.02192,2.07749,0.412302,0.707765
4,4,0.0,0.0,0.0,0.0,0.0,36.5,108.5,280.0,280.0,210.0,140.937,76.2317,64.1437,134.249,159.57,133.664,59.4242,53.169,145.433,106.338,173.581,54.7328,182.964,100.083,70.3708,98.5191,40.6587,39.0949,75.0622,68.807,93.8277,95.3915,106.338,15.0,0.0,10.0,0.0,10.0,0.0,20.0,0.0,5.0,0.0,0.0,-0.141418,0.20094,-0.121316,0.0593259,-0.196599,-0.346025,-0.208411,-0.0198977,-0.0716557,0.134316,0.0857661,0.0959584,0.532858,1.182,1.14408,1.73286,1.97577,0.701913,0.459916,2.02461,2.08874,0.358659,0.69147
5,5,0.0,0.0,0.0,0.0,0.0,0.5,72.5,280.0,280.0,210.0,150.97,55.8508,102.22,145.507,160.944,165.303,59.4242,53.169,145.433,106.338,173.581,54.7328,182.964,100.083,70.3708,98.5191,40.6587,39.0949,75.0622,68.807,93.8277,95.3915,106.338,15.0,0.0,10.0,0.0,10.0,0.0,20.0,0.0,5.0,0.0,0.0,-0.150896,0.209007,-0.135164,0.0607371,-0.211186,-0.364981,-0.227367,-0.038114,-0.0913511,0.100176,0.0535606,0.0566445,0.473524,1.10646,1.05955,1.67242,1.92738,0.628362,0.397342,1.98826,2.04034,0.307063,0.657734
6,6,0.0,0.0,0.0,0.0,0.0,36.5,36.5,280.0,280.0,204.607,124.84,103.851,134.039,131.112,157.023,161.828,60.4314,54.0702,147.898,108.14,176.523,55.6605,186.065,101.779,71.5635,100.189,41.3478,39.7575,76.3344,69.9732,95.418,97.0083,108.14,15.0,0.0,10.0,0.0,10.0,0.0,20.0,0.0,5.0,0.0,0.0,-0.154829,0.234633,-0.1373,0.0410583,-0.219045,-0.374173,-0.234226,-0.0370766,-0.100593,0.091204,0.0543231,0.0561735,0.446313,1.11192,1.0177,1.63554,1.8941

In [34]:
#save result_df to csv
CSV.write("results/market_clearing_nodal_angles.csv", result_df)

"results/market_clearing_nodal_angles.csv"

In [35]:
#claculate capacities for each transmission line transmission lines as column and hours as rows
transmission_line_capacities = DataFrame(hour = Int[])

#add for each line a column
for line in 1:L
    transmission_line_capacities[!, Symbol("line_capacity_$line")] = Float64[]
end

for hour in 1:24
    hour_results = []

    for line in 1:L
        push!(hour_results, 500*(value(angle[transmission_lines[line, 1],hour])-value(angle[transmission_lines[line, 2],hour])))
    end
    push!(transmission_line_capacities, [hour; hour_results])
end

#add the capcities from transmission_lines as a row below
cap = []
push!(cap, 99999)
for line in 1:L
    push!(cap, transmission_lines[line, 4])
end
push!(transmission_line_capacities, cap)


println(transmission_line_capacities)  


25×35 DataFrame
 Row │ hour   line_capacity_1  line_capacity_2  line_capacity_3  line_capacity_4  line_capacity_5  line_capacity_6  line_capacity_7  line_capacity_8  line_capacity_9  line_capacity_10  line_capacity_11  line_capacity_12  line_capacity_13  line_capacity_14  line_capacity_15  line_capacity_16  line_capacity_17  line_capacity_18  line_capacity_19  line_capacity_20  line_capacity_21  line_capacity_22  line_capacity_23  line_capacity_24  line_capacity_25  line_capacity_26  line_capacity_27  line_capacity_28  line_capacity_29  line_capacity_30  line_capacity_31  line_capacity_32  line_capacity_33  line_capacity_34 
     │ Int64  Float64          Float64          Float64          Float64          Float64          Float64          Float64          Float64          Float64          Float64           Float64           Float64           Float64           Float64           Float64           Float64           Float64           Float64           Float64           Float64           Fl

In [36]:
#claculate capacities for each transmission line transmission lines as column and hours as rows
transmission_line_capacities = DataFrame(hour = Int[])

#add for each line a column
for line in 1:L
    transmission_line_capacities[!, Symbol("line_capacity_$line")] = Float64[]
end

for hour in 1:24
    hour_results = []

    for line in 1:L
        n=44+transmission_lines[line, 1]
        m=44+transmission_lines[line, 2]
        push!(hour_results, 500*(result_df[hour, n] -result_df[hour, m]))
    end
    push!(transmission_line_capacities, [hour; hour_results])
end

#add the capcities from transmission_lines as a row below
cap = []
push!(cap, 99999)
for line in 1:L
    push!(cap, transmission_lines[line, 4])
end
push!(transmission_line_capacities, cap)


println(transmission_line_capacities)  


25×35 DataFrame
 Row │ hour   line_capacity_1  line_capacity_2  line_capacity_3  line_capacity_4  line_capacity_5  line_capacity_6  line_capacity_7  line_capacity_8  line_capacity_9  line_capacity_10  line_capacity_11  line_capacity_12  line_capacity_13  line_capacity_14  line_capacity_15  line_capacity_16  line_capacity_17  line_capacity_18  line_capacity_19  line_capacity_20  line_capacity_21  line_capacity_22  line_capacity_23  line_capacity_24  line_capacity_25  line_capacity_26  line_capacity_27  line_capacity_28  line_capacity_29  line_capacity_30  line_capacity_31  line_capacity_32  line_capacity_33  line_capacity_34 
     │ Int64  Float64          Float64          Float64          Float64          Float64          Float64          Float64          Float64          Float64          Float64           Float64           Float64           Float64           Float64           Float64           Float64           Float64           Float64           Float64           Float64           Fl